# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [2]:
import numpy as np

In [5]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [8]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.967400398506522, 10.0)

In [3]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    
    U = np.random.uniform(size=n_sample)*mat
    default_time = -(1/def_rate)*np.log(U)
    bond_price = np.exp(-rf_rate*default_time)*recovery
    bond_price[default_time>1] = np.exp(-rf_rate*mat)
    price = bond_price.mean()
    ### <-- YOUR ANSWER HERE
    ### <-- YOUR ANSWER HERE
    ### <-- YOUR ANSWER HERE
    return price

# Call your function
corp_bond(1, 0.03, 0.04, 0.3, 10000)

# Find the mean and std by calling the function 100 times. 
price_list = [corp_bond(1, 0.03, 0.04, 0.3, 10000) for i in range(0,100)]
price_list = np.array(price_list)
[price_list.mean(),price_list.std()]
    
### <-- YOUR ANSWER HERE
### <-- YOUR ANSWER HERE
### <-- YOUR ANSWER HERE


[0.940939958885905, 0.0010339742397603758]

### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [5]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample/2)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.000000000000002, 10.0)

In [4]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    ### <-
    U = np.random.uniform(size=int(n_sample/2))
    default_time = -(1/def_rate)*np.log(U)
    if(antithetic):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
    bond_price = np.exp(-rf_rate*default_time)*recovery
    bond_price[default_time>1] = np.exp(-rf_rate*mat)
    price = bond_price.mean()

    ### <--
    return price

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
price_list1 = [corp_bond_cv(1, 0.03, 0.04, 0.3, 10000, True, False) for i in range(0,100)]
price_list2 = [corp_bond_cv(1, 0.03, 0.04, 0.3, 10000, False, True) for i in range(0,100)]
price_list3 = [corp_bond_cv(1, 0.03, 0.04, 0.3, 10000, True, True) for i in range(0,100)]
price_list1 = np.array(price_list1)
price_list2 = np.array(price_list2)
price_list3 = np.array(price_list3)
[price_list1.mean(),price_list1.std(),price_list2.mean(),price_list2.std(),price_list3.mean(),price_list3.std()]
### <-- YOUR ANSWER HERE
### <-- YOUR ANSWER HERE
### <-- YOUR ANSWER HERE

[0.9407986701187905,
 0.0010971025066141037,
 0.9416057510611171,
 0.009080424975474757,
 0.9406345396926672,
 0.004411775984100305]

### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [10]:
### Put the analytic expression for the corporate bond price
mat=1
def_rate=0.03
rf_rate=0.04 
recovery=0.3
n_sample=10000

price_acv = def_rate*((-1/(def_rate+rf_rate)*np.exp(-(def_rate+rf_rate))+1/(def_rate+rf_rate))*recovery+1/def_rate*np.exp(-(def_rate+rf_rate)))

price_mc = price_list.mean()
price_diff = price_acv - price_mc
price_diff 
### <-- YOUR ANSWER HERE
### <-- YOUR ANSWER HERE

0.0001460841749927333

0.9410860430608977